## Authentication and relevant libraries

In [ ]:
import pandas as pd
import pandas_gbq
from pandas_gbq import read_gbq
from google.cloud import bigquery
import numpy as np

from google.auth import default
from google.colab import drive
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

auth.authenticate_user()
print('Authenticated')
creds, _ = default()
gc = gspread.authorize(creds)

Authenticated


# Opportunity Sizing

In [ ]:
#@title Dates for Reference
today = datetime.now()

date_28_days_ago = today - timedelta(days=28)
date_1_month_before = date_28_days_ago - relativedelta(months=1)
date_3_months_before = date_28_days_ago - relativedelta(months=3)

print(f"Date 28 days ago: {date_28_days_ago.strftime('%Y-%m-%d')}; end date must not exceed this date")
print(f"1 month before that date: {date_1_month_before.strftime('%Y-%m-%d')}")
print(f"3 months before that date: {date_3_months_before.strftime('%Y-%m-%d')}")

Date 28 days ago: 2025-09-25; end date must not exceed this date
1 month before that date: 2025-08-25
3 months before that date: 2025-06-25


In [ ]:
#@title Step 1: Input your data - dates can be pick from above, suggested is at least 1 month, best is 3
start_date = '2025-08-25'
end_date = '2025-10-'

app_bundles = ['com.everywear.game5'] # Your Bundle / Bundles Here
kpi_event = ['af_revenue'] #If you know the kpi event(s) the client wants to test, if not, uncomment the commented line on the purchases CTE on step 2 and the query will pick all available kpi events the client is sharing

user_level_table = 'moloco-ods.YOUR_USER.TABLE_NAME' # Create a table name under your dataset in the moloco-ods project

In [ ]:
#@title Step 2: Base User Level Table Query - Warning: this query can take a long time to run

user_level_query = f"""
CREATE OR REPLACE TABLE `{user_level_table}` AS

WITH installs AS (
  SELECT
    CASE
    WHEN `moloco-ml.lat_utils.is_userid_truly_available`(device.idfa) THEN device.idfa
    WHEN `moloco-ml.lat_utils.is_userid_truly_available`(mmp.device_id) THEN mmp.device_id
    WHEN `moloco-ml.lat_utils.is_userid_truly_available`(device.idfv) THEN device.idfv
    END AS user_id,
    CASE
      WHEN COALESCE(attribution.attributed, FALSE) THEN "ATTRIBUTED"
      WHEN COALESCE(attribution.organic, FALSE) THEN "ORGANIC"
      ELSE "UNATTRIBUTED"
    END AS attribution,
    mmp.name mmp_name,
    app.bundle as app_bundle,
    device.os,
    device.country,
    IF(event.event_at <> '1970-01-01 00:00:00 UTC', event.event_at, timestamp) as install_at,
    IF(timestamp <> '1970-01-01 00:00:00 UTC', timestamp, timestamp) as ts_install_at
  FROM
    `focal-elf-631.df_accesslog.pb`
  WHERE DATE(timestamp) >= '{start_date}' AND DATE(timestamp) <= '{end_date}'
    AND device.os = "ANDROID"
    AND app.bundle IN UNNEST({app_bundles})
    AND event.name = "install"
),


purchases as (
  SELECT
    CASE
      WHEN `moloco-ml.lat_utils.is_userid_truly_available`(device.idfa) THEN device.idfa
      WHEN `moloco-ml.lat_utils.is_userid_truly_available`(mmp.device_id) THEN mmp.device_id
      WHEN `moloco-ml.lat_utils.is_userid_truly_available`(device.idfv) THEN device.idfv
    END AS user_id,
    mmp.name mmp_name,
    device.os,
    app.bundle as app_bundle,
    event.name event_name,
    IF(event.event_at <> '1970-01-01 00:00:00 UTC', event.event_at, timestamp) as event_at,
    event.revenue_usd.amount as revenue
  FROM `focal-elf-631.df_accesslog.pb`
  WHERE DATE(timestamp) >= '{start_date}'
    AND DATE(timestamp) <= DATE_ADD('{end_date}', INTERVAL 28 DAY)
    AND device.os = "ANDROID"
    AND app.bundle IN UNNEST({app_bundles})
    AND event.name IN UNNEST({kpi_event})
    --AND NOT REGEXP_CONTAINS(LOWER(event.name), r'(?i)(ad_|_ad|adrev|af_ad_clicked|ADMON|max_ilrd|impression|iaa|ads_rev|ads_reward|install|reengagement|reattribution|af_app_opened|rejected_install|session|__SESSION__|app_open|login|app_open|Delete Account|delete|rejected_inst|first_open|open)')
),

platform_info AS (
  SELECT
    DISTINCT
    platform_id,
    advertiser.mmp_bundle_id AS app_bundle,
    product.genre AS genre
  FROM `moloco-ae-view.athena.fact_dsp_core`
  WHERE date_utc >= '{start_date}' AND date_utc <= '{end_date}'
    AND advertiser.mmp_bundle_id IN UNNEST({app_bundles})
)


SELECT
  user_id,
  attribution,
  pi.platform_id,
  mmp_name,
  app_bundle,
  pi.genre,
  i.os,
  country,
  TIMESTAMP_DIFF(event_at, i.install_at, DAY) AS days_since_install,
  TIMESTAMP_DIFF(event_at, i.install_at, HOUR) AS hours_since_install,
  i.install_at,
  event_at,
  p.event_name,
  revenue as event_pb_rev
FROM
  installs i
LEFT JOIN
  purchases p
USING(user_id, app_bundle, mmp_name)
INNER JOIN platform_info pi USING (app_bundle)
WHERE user_id is not null
AND TIMESTAMP_DIFF(event_at, install_at, DAY) >=0
AND TIMESTAMP_DIFF(event_at, install_at, HOUR) >=0
AND TIMESTAMP_DIFF(event_at, install_at, DAY) < 28
"""


In [ ]:
try:
    pandas_gbq.read_gbq(user_level_query, project_id="moloco-interactive")
except KeyError:
    # This is expected since CREATE TABLE doesn't return columns
    pass

print(f"Table {user_level_table} created successfully!")

Table moloco-ods.jacqueline.d28_oppty_sizing_cpa_metacore_everywear created successfully!


In [ ]:
#@title Step 3: Aggregate Base query results - Run this after your table has been created in Step 2
agg_results_query = f"""
WITH payer_cat AS (
  SELECT
    user_id,
    app_bundle,
    event_name,
    MIN(TIMESTAMP_DIFF(event_at, install_at, DAY)) AS fp_days_since_install,
    CASE
      WHEN MIN(TIMESTAMP_DIFF(event_at, install_at, DAY)) < 7 THEN "d7_payer"
      WHEN MIN(TIMESTAMP_DIFF(event_at, install_at, DAY)) >= 7 AND MIN(TIMESTAMP_DIFF(event_at, install_at, DAY)) <28 THEN "d8_d28_payer"
      WHEN MIN(TIMESTAMP_DIFF(event_at, install_at, DAY)) >= 28 THEN 'd28_plus_payer'
    END AS payer_type
  FROM `{user_level_table}`
  GROUP BY ALL
)


SELECT
  platform_id,
  mmp_name,
  app_bundle,
  country,
  attribution,
  event_name,

  SUM(CASE WHEN days_since_install < 7 THEN event_pb_rev ELSE 0 END) AS d7_total_revenue,
  COUNT(CASE WHEN days_since_install < 7 THEN user_id END) AS d7_total_actions,
  COUNT(DISTINCT CASE WHEN days_since_install < 7 THEN user_id END) AS d7_total_payers,

  SUM(IF(days_since_install < 28, event_pb_rev, 0)) AS d28_total_revenue,
  COUNT(CASE WHEN days_since_install < 28 THEN user_id END) AS d28_total_actions,
  COUNT(DISTINCT CASE WHEN days_since_install < 28 THEN user_id END) AS d28_total_payers,

  SUM(IF(payer_type = 'd7_payer', event_pb_rev, 0)) as d7_payers_revenue,
  SUM(IF(payer_type = 'd7_payer' AND days_since_install < 7, event_pb_rev, 0)) as d7_payers_revenue,
  COUNT(IF(payer_type = 'd7_payer' AND days_since_install < 7, user_id,NULL)) as d7_actions,
  COUNT(DISTINCT IF(payer_type = 'd7_payer', user_id, NULL)) as d7_payers,

  SUM(CASE WHEN days_since_install >= 7 AND days_since_install < 28 THEN event_pb_rev ELSE 0 END) AS d8_d28_payers_revenue,
  COUNT(CASE WHEN days_since_install >= 7 AND days_since_install < 28 THEN user_id END) AS d8_d28_actions,
  COUNT(DISTINCT CASE WHEN days_since_install >= 7 AND days_since_install < 28 THEN user_id END) AS d8_d28_payers,

  SUM(IF(payer_type = 'd7_payer' AND days_since_install >= 7 AND days_since_install < 28, event_pb_rev, 0)) as d7_payers_revenue_extra_d8_d28,
  COUNT(IF(payer_type = 'd7_payer' AND days_since_install >= 7 AND days_since_install < 28, user_id, NULL)) as d7_payers_actions_extra_d8_d28,

  SUM(IF(payer_type = 'd8_d28_payer', event_pb_rev, 0)) as d8_d28_payers_revenue_extra,
  COUNT(IF(payer_type = 'd8_d28_payer', user_id, NULL)) as d8_d28_actions_extra,
  COUNT(DISTINCT IF(payer_type = 'd8_d28_payer', user_id, NULL)) as d8_d28_payers_extra,

  SAFE_DIVIDE(COUNT(DISTINCT IF(payer_type = 'd8_d28_payer', user_id, NULL)), COUNT(DISTINCT CASE WHEN days_since_install < 28 THEN user_id END)) AS d8_d28_extra_payers_pct
FROM `{user_level_table}`
LEFT JOIN payer_cat USING(user_id, app_bundle, event_name)
GROUP BY ALL
-- HAVING COUNT(DISTINCT CASE WHEN days_since_install < 28 THEN user_id END) >=100
ORDER BY 1,2,3
"""

df_agg = pandas_gbq.read_gbq(agg_results_query, project_id='moloco-interactive')

Downloading: 100%|██████████|


In [ ]:
df_agg.columns

Index(['platform_id', 'mmp_name', 'app_bundle', 'country', 'attribution',
       'event_name', 'd7_total_revenue', 'd7_total_actions', 'd7_total_payers',
       'd28_total_revenue', 'd28_total_actions', 'd28_total_payers',
       'd7_payers_revenue', 'd7_payers_revenue_1', 'd7_actions', 'd7_payers',
       'd8_d28_payers_revenue', 'd8_d28_actions', 'd8_d28_payers',
       'd7_payers_revenue_extra_d8_d28', 'd7_payers_actions_extra_d8_d28',
       'd8_d28_payers_revenue_extra', 'd8_d28_actions_extra',
       'd8_d28_payers_extra', 'd8_d28_extra_payers_pct'],
      dtype='object')

## Results

In [ ]:
#@title [Optional] Filter: Over 100 D28 Total Payers and > 0% Extra Payers in D8-D28. If testing multiple/alternative events to the BAU, this step it to make sure those have enough volume for our models to optimise
df_agg_filtered = df_agg.loc[df_agg.d8_d28_extra_payers_pct != 0]
df_agg_filtered = df_agg_filtered.loc[df_agg_filtered.d28_total_payers >= 100]

df_agg_filtered.sort_values('d8_d28_extra_payers_pct', ascending=False)

,platform_id,mmp_name,app_bundle,country,attribution,event_name,d7_total_revenue,d7_total_actions,d7_total_payers,d28_total_revenue,...,d7_payers,d8_d28_payers_revenue,d8_d28_actions,d8_d28_payers,d7_payers_revenue_extra_d8_d28,d7_payers_actions_extra_d8_d28,d8_d28_payers_revenue_extra,d8_d28_actions_extra,d8_d28_payers_extra,d8_d28_extra_payers_pct
4,METACOREGAMES,APPSFLYER,com.everywear.game5,CAN,UNATTRIBUTED,af_revenue,8.601452e+06,936448,76,2.387524e+07,...,76,1.527379e+07,1575296,99,1.270568e+07,1218176,2.568111e+06,357120,47,0.382114
7,METACOREGAMES,APPSFLYER,com.everywear.game5,JPN,UNATTRIBUTED,af_revenue,3.326086e+07,3178368,275,1.004935e+08,...,275,6.723264e+07,5670272,348,5.474222e+07,4138624,1.249042e+07,1531648,170,0.382022
1,METACOREGAMES,APPSFLYER,com.everywear.game5,DEU,UNATTRIBUTED,af_revenue,2.487420e+07,2468096,247,9.243741e+07,...,247,6.756321e+07,5698048,296,5.594525e+07,4364800,1.161796e+07,1333248,140,0.361757
14,METACOREGAMES,APPSFLYER,com.everywear.game5,FRA,UNATTRIBUTED,af_revenue,1.437403e+07,1333248,124,3.692076e+07,...,124,2.254674e+07,2110976,142,1.672904e+07,1488000,5.817693e+06,622976,68,0.354167
5,METACOREGAMES,APPSFLYER,com.everywear.game5,KOR,UNATTRIBUTED,af_revenue,1.178446e+07,1241984,115,4.213626e+07,...,115,3.035181e+07,2702208,140,2.556688e+07,2134784,4.784922e+06,567424,62,0.350282
36,METACOREGAMES,APPSFLYER,com.everywear.game5,GBR,UNATTRIBUTED,af_revenue,2.234754e+07,2718080,230,6.030901e+07,...,230,3.796147e+07,4325120,244,3.087983e+07,3233920,7.081639e+06,1091200,111,0.325513
90,METACOREGAMES,APPSFLYER,com.everywear.game5,USA,UNATTRIBUTED,af_revenue,1.188742e+08,13737216,1142,3.630663e+08,...,1142,2.441921e+08,22292224,1189,1.996483e+08,17070336,4.454377e+07,5221888,545,0.323059


In [ ]:
#@title Final Step: import dataframe to Google Sheet
# Create spreadsheet with today's date in the name
todays_date = today.strftime('%Y-%m-%d')
spreadsheet_name = f'D28_Opportunity_Sizing_For_{app_bundles}_{todays_date}'

# Create new Google Sheet
spreadsheet = gc.create(spreadsheet_name)

# Write your data to the sheet
worksheet = spreadsheet.sheet1
set_with_dataframe(worksheet, df_agg_filtered.sort_values('d8_d28_extra_payers_pct', ascending=False))

print(f"Click here to open: {spreadsheet.url}")

Click here to open: https://docs.google.com/spreadsheets/d/17br8jR805YF97A9WkdjzFmyPMtkBrwoSjvaTg0xVfE4


## With the data above, focus on the d8_d28_extra_payers_pct column following the methodology benchmark here to asses the eligibility of the bundle x kpi event x geo: [D28 CPA deck](https://docs.google.com/presentation/d/1Lmy8es3UT1JqOjOyksRQ8XQKFSvvuEWUCPe2NzFrJNk/edit?slide=id.g3706bd4ec6b_0_23#slide=id.g3706bd4ec6b_0_23)